In [1]:
import pandas as pd

In [2]:
data=pd.read_excel('Extract2.xlsx')

In [5]:
cvs=data.groupby(['OBJECT_NAME'])

In [7]:
cvs.groups.keys()

dict_keys(['CV_CLAIM', 'CV_GROUP_VIEW'])

In [10]:
cv_claim=cvs.get_group('CV_CLAIM')

In [12]:
cv_claim['TARGET_NODE']

0      Projection_1
1      Projection_1
2      Projection_1
3      Projection_1
4      Projection_1
           ...     
137    logicalModel
138          Join_5
139          Join_5
140             NaN
141             NaN
Name: TARGET_NODE, Length: 132, dtype: object

In [14]:
cv_claim['SOURCE_NODE'].unique()

array(['CLAIM', 'CV_GROUP_VIEW', 'CLAIMANT', 'CLAIM_STATUS',
       '#Projection_1', '#Projection_2', '#Projection_3', '#Join_1',
       '#Join_2', '#Projection_4', '#Join_3', '#Join_4', 'Join_5', nan],
      dtype=object)

In [15]:
cv_claim['TARGET_NODE'].unique()

array(['Projection_1', 'Projection_2', 'Projection_3', 'Projection_4',
       'Join_1', 'Join_2', 'Join_3', 'Join_4', 'Join_5', 'logicalModel',
       nan], dtype=object)

In [20]:
cv_claim['SOURCE_NODE']=cv_claim['SOURCE_NODE'].fillna('')

C:\Users\imsud\AppData\Local\Temp/ipykernel_3768/1347838216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_claim['SOURCE_NODE']=cv_claim['SOURCE_NODE'].fillna('')


In [40]:
cv_claim['SOURCE_NODE']=cv_claim['SOURCE_NODE'].replace('#','',regex=True)

C:\Users\imsud\AppData\Local\Temp/ipykernel_3768/3848344398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_claim['SOURCE_NODE']=cv_claim['SOURCE_NODE'].replace('#','',regex=True)


In [25]:
cv_claim['TARGET_NODE']=cv_claim['TARGET_NODE'].fillna('')

C:\Users\imsud\AppData\Local\Temp/ipykernel_3768/1868942600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_claim['TARGET_NODE']=cv_claim['TARGET_NODE'].fillna('')


In [41]:
cv_claim

,OBJECT_NAME,NODE_ID,TARGET_NODE,TARGET_VALUE,SOURCE_NODE,SOURCE_VALUE,SOURCE_TYPE,PROMPT_NAME,DATATYPE,LEN,MANDATORY,FORMULA
0,CV_CLAIM,110.0,Projection_1,CLAIM_UID,CLAIM,CLAIM_UID,DATA_BASE_TABLE,NaN,NaN,NaN,NaN,NaN
1,CV_CLAIM,110.0,Projection_1,CLIENT_NUMBER,CLAIM,CLIENT_NUMBER,DATA_BASE_TABLE,NaN,NaN,NaN,NaN,NaN
2,CV_CLAIM,110.0,Projection_1,LINE_OF_BUSINESS,CLAIM,LINE_OF_BUSINESS,DATA_BASE_TABLE,NaN,NaN,NaN,NaN,NaN
3,CV_CLAIM,110.0,Projection_1,STATE_OF_LOSS,CLAIM,STATE_OF_LOSS,DATA_BASE_TABLE,NaN,NaN,NaN,NaN,NaN
4,CV_CLAIM,110.0,Projection_1,FORMATTED_FILE_NUMBER,CLAIM,FORMATTED_FILE_NUMBER,DATA_BASE_TABLE,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
137,CV_CLAIM,814.0,logicalModel,CA_COUNT,Join_5,CA_COUNT,NaN,NaN,NaN,NaN,NaN,NaN
138,CV_CLAIM,814.0,Join_5,CA_DATE_OF_LOSS,,NaN,NaN,NaN,DATE,NaN,NaN,"daydate(""DATE_OF_LOSS"")"
139,CV_CLAIM,814.0,Join_5,CA_COUNT,,NaN,NaN,NaN,INTEGER,NaN,NaN,countDistinct( FORMATTED_FILE_NUMBER )
140,CV_CLAIM,NaN,,NaN,,NaN,NaN,IP_GROUP,NVARCHAR,150.0,1.0,NaN


In [49]:
edges=list(cv_claim.groupby(['SOURCE_NODE','TARGET_NODE']).groups.keys())[2:]

In [50]:
edges

[('CLAIM', 'Projection_1'),
 ('CLAIMANT', 'Projection_3'),
 ('CLAIM_STATUS', 'Projection_4'),
 ('CV_GROUP_VIEW', 'Projection_2'),
 ('Join_1', 'Join_3'),
 ('Join_2', 'Join_3'),
 ('Join_3', 'Join_5'),
 ('Join_4', 'Join_5'),
 ('Join_5', 'logicalModel'),
 ('Projection_1', 'Join_1'),
 ('Projection_2', 'Join_1'),
 ('Projection_2', 'Join_2'),
 ('Projection_2', 'Join_4'),
 ('Projection_3', 'Join_2'),
 ('Projection_4', 'Join_4')]

In [58]:
nodes=list(pd.concat([cv_claim['SOURCE_NODE'],cv_claim['TARGET_NODE']]).unique())

In [60]:
nodes.remove('')

In [61]:
nodes

['CLAIM',
 'CV_GROUP_VIEW',
 'CLAIMANT',
 'CLAIM_STATUS',
 'Projection_1',
 'Projection_2',
 'Projection_3',
 'Join_1',
 'Join_2',
 'Projection_4',
 'Join_3',
 'Join_4',
 'Join_5',
 'logicalModel']

In [63]:
def get_nodes_dict(nodes):
    output=[]
    for node in nodes:
        output.append({'data':{'id':node,'label':node}})
    return output

In [67]:
final_nodes=get_nodes_dict(nodes)

In [65]:
def get_edges_dict(edges):
    output=[]
    for edge in edges:
        output.append({'data':{'source':edge[0],'target':edge[1]}})
    return output

In [68]:
final_edges=get_edges_dict(edges)

In [69]:
elements=final_nodes+final_edges

In [70]:
elements

[{'data': {'id': 'CLAIM', 'label': 'CLAIM'}},
 {'data': {'id': 'CV_GROUP_VIEW', 'label': 'CV_GROUP_VIEW'}},
 {'data': {'id': 'CLAIMANT', 'label': 'CLAIMANT'}},
 {'data': {'id': 'CLAIM_STATUS', 'label': 'CLAIM_STATUS'}},
 {'data': {'id': 'Projection_1', 'label': 'Projection_1'}},
 {'data': {'id': 'Projection_2', 'label': 'Projection_2'}},
 {'data': {'id': 'Projection_3', 'label': 'Projection_3'}},
 {'data': {'id': 'Join_1', 'label': 'Join_1'}},
 {'data': {'id': 'Join_2', 'label': 'Join_2'}},
 {'data': {'id': 'Projection_4', 'label': 'Projection_4'}},
 {'data': {'id': 'Join_3', 'label': 'Join_3'}},
 {'data': {'id': 'Join_4', 'label': 'Join_4'}},
 {'data': {'id': 'Join_5', 'label': 'Join_5'}},
 {'data': {'id': 'logicalModel', 'label': 'logicalModel'}},
 {'data': {'source': 'CLAIM', 'target': 'Projection_1'}},
 {'data': {'source': 'CLAIMANT', 'target': 'Projection_3'}},
 {'data': {'source': 'CLAIM_STATUS', 'target': 'Projection_4'}},
 {'data': {'source': 'CV_GROUP_VIEW', 'target': 'Project

In [71]:
import pickle

In [74]:
with open('elements.pkl','w') as f:
    pickle.dump(elements,f)

TypeError: write() argument must be str, not bytes